In [0]:
%pip install mlflow==2.10.1 langchain==0.1.5 databricks-vectorsearch==0.22 databricks-sdk==0.18.0 mlflow[databricks]
dbutils.library.restartPython()

In [0]:
import os
host = "https://" + spark.conf.get("spark.databricks.workspaceUrl")
os.environ["DATABRICKS_TOKEN"] = dbutils.secrets.get(scope="project-dev", key="databricks-token")

index_name = "workspace.input_data.drug_data_vector_index"
VECTOR_SEARCH_ENDPOINT_NAME = "vector_search_endpoint"

In [0]:
from databricks.vector_search.client import VectorSearchClient
from langchain_community.vectorstores import DatabricksVectorSearch
from langchain_community.embeddings import DatabricksEmbeddings

embedding_model = DatabricksEmbeddings(endpoint="databricks-qwen3-embedding-0-6b")

def get_retriever(persist_dir: str = None):
    os.environ["DATABRICKS_HOST"] = host
    # get the vector search index
    vsc = VectorSearchClient(workspace_url=host, personal_access_token=os.environ["DATABRICKS_TOKEN"])
    vs_index = vsc.get_index(
        endpoint_name=VECTOR_SEARCH_ENDPOINT_NAME,
        index_name=index_name
    )
    # create the retriever
    vectorstore = DatabricksVectorSearch(
        vs_index,
        text_column = "chunk_text",
        embedding=embedding_model
    )
    return vectorstore.as_retriever()

In [0]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatDatabricks

chat_model = ChatDatabricks(endpoint="databricks-qwen3-next-80b-a3b-instruct", max_tokens = 200)

TEMPLATE = """You are an assistant that helps users find information about the drugs. If the question is not related to drugs, kindly decline to answer. If you don't know the answer, just say you don't know, don't try to make-up the answer. If the question is related to the drug you don't have data for just say its out of your knowledge base. Keep the answer as concise as possible. Provide answers in english only.
Use the following pieces of context to answer the question at the end:
{context}
Question: {question}
Answer:
"""
prompt = PromptTemplate(template=TEMPLATE, input_variables=["context", "question"])

chain = RetrievalQA.from_chain_type(
    llm=chat_model,
    chain_type="stuff",
    retriever=get_retriever(),
    chain_type_kwargs={"prompt": prompt}
    )#return_source_documents=True,

In [0]:
question = {"query":"what are the uses of paracetamol and metformin ?"}
answer= chain.invoke(question)
print(answer)

In [0]:
from mlflow.models import infer_signature
import mlflow
import langchain

mlflow.set_registry_uri("databricks-uc")
model_name = "workspace.rag.drug_chatbot_model"

with mlflow.start_run(run_name="appliance_chatbot_run") as run:
    signature = infer_signature(question, answer)
    model_info = mlflow.langchain.log_model(
        chain,
        loader_fn = get_retriever,
        artifact_path = 'chain',
        registered_model_name = model_name,
        pip_requirements=[
            "mlflow==" + mlflow.__version__,
            "langchain==" + langchain.__version__,
            "databricks-vectorsearch"
        ],
        input_example=question,
        signature=signature
    )